In [217]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
pd.set_option('display.max_columns', None)

In [218]:
df= pd.read_csv("../data/chicago_data_portal_csv_files/Chicago_Public_Schools_-_School_Profile_Information_SY1819.csv")
df= df[df['Is_High_School']==True]

In [219]:
df.head()

,School_ID,Legacy_Unit_ID,Finance_ID,Short_Name,Long_Name,Primary_Category,Is_High_School,Is_Middle_School,Is_Elementary_School,Is_Pre_School,Summary,Administrator_Title,Administrator,Secondary_Contact_Title,Secondary_Contact,Address,City,State,Zip,Phone,Fax,CPS_School_Profile,Website,Facebook,Twitter,Youtube,Pinterest,Attendance_Boundaries,Grades_Offered_All,Grades_Offered,Student_Count_Total,Student_Count_Low_Income,Student_Count_Special_Ed,Student_Count_English_Learners,Student_Count_Black,Student_Count_Hispanic,Student_Count_White,Student_Count_Asian,Student_Count_Native_American,Student_Count_Other_Ethnicity,Student_Count_Asian_Pacific_Islander,Student_Count_Multi,Student_Count_Hawaiian_Pacific_Islander,Student_Count_Ethnicity_Not_Available,Statistics_Description,Demographic_Description,Dress_Code,PreK_School_Day,Kindergarten_School_Day,School_Hours,Freshman_Start_End_Time,After_School_Hours,Earliest_Drop_Off_Time,Classroom_Languages,Bilingual_Services,Refugee_Services,Title_1_Eligible,PreSchool_Inclusive,Preschool_Instructional,Significantly_Modified,Hard_Of_Hearing,Visual_Impairments,Transportation_Bus,Transportation_El,Transportation_Metra,School_Latitude,School_Longitude,Average_ACT_School,Mean_ACT,College_Enrollment_Rate_School,College_Enrollment_Rate_Mean,Graduation_Rate_School,Graduation_Rate_Mean,Overall_Rating,Rating_Status,Rating_Statement,Classification_Description,School_Year,Third_Contact_Title,Third_Contact_Name,Fourth_Contact_Title,Fourth_Contact_Name,Fifth_Contact_Title,Fifth_Contact_Name,Sixth_Contact_Title,Sixth_Contact_Name,Seventh_Contact_Title,Seventh_Contact_Name,Network,Is_GoCPS_Participant,Is_GoCPS_PreK,Is_GoCPS_Elementary,Is_GoCPS_High_School,Open_For_Enrollment_Date,Closed_For_Enrollment_Date
0,400172,9647,66255,ASPIRA - BUSINESS & FINANCE HS,ASPIRA Business and Finance,HS,True,False,False,False,The ASPIRA Business and Finance High School ta...,Director,Dr. Hector Freytas,Assistant Principal,Veronica Galindo,2989 N MILWAUKEE AVE,Chicago,Illinois,60618,7.732521e+09,NaN,http://cps.edu/Schools/Pages/school.aspx?Schoo...,http://www.aspirail.org/ASPIRASchools,https://www.facebook.com/ASPIRABusiness,https://twitter.com/ASPIRABusiness,NaN,NaN,False,"9,10,11,12",9-12,633,414,130,195,17,597,10,4,1,0,0,4,0,0,There are 633 students enrolled at ASPIRA - BU...,The largest demographic at ASPIRA - BUSINESS &...,True,NaN,NaN,7:30AM-4:30PM,NaN,3:15PM-4:30PM,7:30AM,"Spanish, Spanish for Heritage Speakers",True,NaN,True,NaN,NaN,NaN,NaN,NaN,56,Blue,NaN,41.935726,-87.718204,NaN,NaN,NaN,68.2,NaN,78.2,Level 2+,NOT APPLICABLE,"This school received a Level 2+ rating, which ...","Schools that are open to all Chicago children,...",School Year 2018-2019,Dean of Students,Mike Simball,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Charter,True,False,False,True,07/01/2015 12:00:00 AM,NaN
2,609780,2090,41111,MARINE LEADERSHIP AT AMES HS,Marine Leadership Academy at Ames,HS,True,True,False,False,Marine Leadership Academy prepares leaders in ...,Principal,Mrs.Erin A Galfer,Other,Juan Carlos Zayas,1920 N HAMLIN AVE,Chicago,Illinois,60647,7.735345e+09,7.735345e+09,http://cps.edu/Schools/Pages/school.aspx?Schoo...,http://marine.cps.edu,https://www.facebook.com/marineleadershipacademy,NaN,NaN,NaN,False,"7,8,9,10,11,12",7-12,847,825,79,158,17,817,8,2,1,0,0,2,0,0,There are 847 students enrolled at MARINE LEAD...,The largest demographic at MARINE LEADERSHIP A...,True,NaN,NaN,07:45 AM-03:25 PM,NaN,NaN,NaN,Spanish,True,NaN,True,NaN,NaN,NaN,NaN,NaN,73,NaN,NaN,41.916045,-87.721742,NaN,NaN,59.2,68.2,NaN,78.2,Level 1,GOOD STANDING,"This school received a Level 1 rating, which i...",For students who wish to develop leadership an...,School Year 2018-2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ISP,True,False,True,True,09/01/2004 12:00:00 AM,NaN
5,609750,1750,49051,SIMPSON HS,Simpson Academy HS for Young Women,HS,True,True,True,False,``,Principal,Mrs.Sherita D Carter-King,Guidance Counselor,Veronica Falls,1321 S PAULINA ST,Chicago,Illinois,60608,7.735348e+09,7

In [220]:
df.dropna(subset=['Graduation_Rate_School'],  inplace=True)

## Train Test Split: Holdout set of .20


In [244]:
from sklearn.model_selection import train_test_split
X = df.drop("Graduation_Rate_School", axis=1)
y = df["Graduation_Rate_School"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

## Model: Graduation_Rate_School ~ Student_Count_Total + Network 

In [222]:
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

In [250]:
X_train.Network.isna().sum()

1

In [251]:
X_train['Student_Count_Total'].isna().sum()

0

In [284]:
X_train['Network_no_na'] = X_train['Network'].fillna('missing')

<ipython-input-284-42bc1317100b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['Network_no_na'] = X_train['Network'].fillna('missing')


In [285]:
categorical_transformer = ColumnTransformer(transformers = [
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False),['Network_no_na'])],remainder='passthrough')

pipe = Pipeline(steps = [
                          ('encoder', categorical_transformer),
                          ('scaler', StandardScaler()),
                          ('regressor', LinearRegression())
                          ])

In [286]:
pipe.fit(X_train[['Network_no_na', 'Student_Count_Total']], y_train)

Pipeline(steps=[('encoder',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('encoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['Network_no_na'])])),
                ('scaler', StandardScaler()),
                ('regressor', LinearRegression())])

In [287]:
pipe.score(X_train[['Network_no_na', 'Student_Count_Total']], y_train)

0.6007137586274484

In [288]:
pipe.predict(X_train[['Network_no_na', 'Student_Count_Total']])

array([ 95.15802142,  56.65802142,  71.09552142,  65.15802142,
        17.15802142,  70.47052142,  90.59552142,  73.09552142,
        74.59552142,  83.90802142,  65.53302142,  75.59552142,
        82.09552142,  15.15802142,  77.59552142,  57.65802142,
        70.84552142,  54.22052142,  69.22052142,  77.34552142,
        17.15802142,  95.15802142,  85.28302142,  82.15802142,
        77.03302142,  71.59552142,  73.28302142,  87.03302142,
        71.78302142,  81.40802142,  79.78302142,  72.28302142,
        94.59552142,  81.47052142,  67.34552142,  71.15802142,
        87.28302142,  82.65802142,  98.28302142,  70.65802142,
        83.90802142,  69.34552142,  77.78302142, 107.09552142,
        96.47052142,  69.53302142,  19.15802142,  56.59552142,
        74.47052142,  17.15802142,  70.03302142,  60.97052142,
        72.84552142,  68.28302142,  67.78302142,  64.15802142,
        88.03302142,  73.47052142,  69.90802142,  78.78302142,
        72.47052142,  86.09552142,  73.09552142, 114.09

## Cross Val


In [289]:
from sklearn.model_selection import cross_val_score

In [290]:
cross_val_score(pipe, X_train[['Network_no_na', 'Student_Count_Total']], y_train)

array([ 4.99979656e-01, -8.48066005e-01,  6.52269431e-01, -5.28024739e+28,
        3.69689306e-01])